In [2]:
import sqlite3
import requests as r
from bs4 import BeautifulSoup as bs
import lxml
import math
import re
import time
import public_crawler
import sys

In [3]:
# 中信房屋爬蟲模組, 其他租屋網可參考此架構
# 可能因為網改而需要隨時調整
# 1. soup.select內容
# 2. post/get爬取網頁本身方法

In [4]:
# 中信房屋 Post form
def getPayload(City, Area, P):
    return {"City": City, "Area": Area, "P": P, "CompartmentStyle": -1, "BuildingType": 0, "Room": 0, "RentalFee": 0, "Footage": 0,  "ViewMode": 1}
#

In [5]:
# 1.抓外頁網址
# 取得 urls, 依據 url 逐筆執行
# !!!!! 睡 1 秒 !!!!!
def innerPageETL_all(dbName, urls):
    i = 0

    # 單筆, 多筆匯入之調整
    urls = [urls] if isinstance(urls, str) else urls
    for url in urls:
        try:

            i = i + innerPageETL_one(dbName, url)

        except Exception as err:
            print('innerPageETL --- Something Error')
            
        time.sleep(1)
    return i
#

In [6]:
# 2.取得 url, 逐筆執行頁內拆解
def innerPageETL_one(dbName, url):
    try:
        title, main, detail = queryByURL(url)
        if (title == '' or isinstance(main, str) or isinstance(detail, str)):
            tick = public_crawler.insertToDB(dbName, url)

        else:
            tick = public_crawler.insertToDB(dbName, url, title, str(main), str(detail))
        
    except Exception as err:
        print(str(err) + ' ; url = ' + url)
        tick = 0

    finally:
        return tick
#

In [7]:
# 3.查詢內頁全部 _ 逐項拆解
def queryByURL(url):
    try:
        res = r.get(url)
        soup = bs(res.text, 'lxml')

        # 內頁標題
        title = extract_title(soup, url)
        # print(title)
        
        # 內頁表格主要資訊
        main = extract_main(soup, url)
        # print(main)
        
        # 內頁表格明細資訊
        detail = extract_detail(soup, url)
        # print(detail)

        return title, main, detail
        
    except Exception as err:
        print(title + " ; 內頁soup.select時Error ; " + url)
#

In [8]:
### 內頁拆解
# 標題
def extract_title(soup, url):
    try:
        title = soup.select('div.content span.f5')[0].text
        
    except Exception as err:
        print('title error : ' + url)
        title = ''
    finally:
        return title
#

In [9]:
# 主要段落
def extract_main(soup, url):
    main = {}
    try:
        for vv in soup.select('div.search_infobg_left3 > ul')[0].select('li'):
            kv = vv.text.strip().replace(' ','').replace('\n', ' ').split('：')
            if len(kv) == 2:
                main[kv[0]] = kv[1]
                
        if soup.select('input#GPSLatitude')[0]['value'] != '':
            main['lat'] = float(soup.select('input#GPSLatitude')[0]['value'])
        if soup.select('input#GPSLongitude')[0]['value'] != '':
            main['lng'] = float(soup.select('input#GPSLongitude')[0]['value'])
                
    except Exception as err:
        print('main error : ' + url)
        main = ''
        
    finally:
        return main
#

In [10]:
# 物件內容
def extract_detail(soup, url):
    ii = 0
    detail = {}
    try:
        for vv in soup.find(lambda tag: tag.name=='table').select('td'):
            if ii == 0:
                kk = vv.text.strip()
            else:
                detail[kk] = vv.text.strip().replace(' ','').replace('\n', ' ')
            ii = 1 if ii == 0 else 0
    
    except Exception as err:
        print('detail error : ' + url)
        detail = ''
    
    finally:
        return detail
#

## test_jc.py

In [11]:
# encoding=utf8
import crawler_sy

dbName = 'sy.db'

        
# 單筆匯入程式
url = "http://rent.sinyi.com.tw/itemDetail.php?s=1&itemid=196321"

i = innerPageETL_all(dbName, url)

print('成功匯入{:3d}筆'.format(i))


title error : http://rent.sinyi.com.tw/itemDetail.php?s=1&itemid=196321
main error : http://rent.sinyi.com.tw/itemDetail.php?s=1&itemid=196321
成功匯入  1筆


In [3]:
# url = "http://rent.cthouse.com.tw/Building/Detail/00236463"
url = "http://rent.sinyi.com.tw/itemDetail.php?s=1&itemid=CA17405"
res = r.get(url)
soup = bs(res.text, 'lxml')

In [5]:
soup.select("script")

[<script>(function(w,d,s,l,i){w[l]={'start':new Date().getTime(),id:i};
 var f=d.getElementsByTagName(s)[0],j=d.createElement(s);j.async=true;
 j.src='//img.sinyi.com/j/'+l+'.js';f.parentNode.insertBefore(j,f);
 })(window,document,'script','sy','595201a101a06d17721dfa83');</script>,
 <script src="http://rent.sinyi.com.tw/js/jquery-1.8.2.min.js"></script>,
 <script src="http://rent.sinyi.com.tw/js/jquery.hoverpulse.js"></script>,
 <script src="http://rent.sinyi.com.tw/js/jquery-ui.min.js"></script>,
 <script src="http://rent.sinyi.com.tw/js/jquery.jcarousellite.pauseOnHover.js"></script>,
 <script src="http://cdn.jquerytools.org/1.2.7/all/jquery.tools.min.js"></script>,
 <script src="http://rent.sinyi.com.tw/js/jquery.filestyle.mini.js"></script>,
 <script src="http://rent.sinyi.com.tw/js/global.js"></script>,
 <script src="http://rent.sinyi.com.tw/js/common.js"></script>,
 <script src="http://rent.sinyi.com.tw/js/jquery-ui-1.7.2.js"></script>,
 <script src="http://rent.sinyi.com.tw/js/

In [12]:
extract_main(soup, url)

{'lat': 25.0043,
 'lng': 121.463,
 '地址': '新北市板橋區重慶路155巷31號5樓',
 '坪數': '5.00坪',
 '押金': '二個月',
 '格局': '1房1衛',
 '樓層': '5樓/共5樓',
 '租金': '\r 6500元/月\r   (含第四台、水費、瓦斯費)',
 '車位': '無',
 '類型': '公寓/獨立套房'}

In [13]:
extract_detail(soup, url)

{'傢俱': '衣櫥、雙人床、書桌、椅子',
 '可遷入日期': '2017/06/21',
 '寵物': '接受',
 '屋齡': '',
 '性別要求': '女',
 '是否接受短期租賃': '否',
 '最短租期': '一年',
 '特色描述': '交通便利--近板南線府中捷運站,步行十分鐘可達 鬧中取靜—板新特區,文教學區 生活機能便利—近百貨公司,傳統市場,大賣場,夜市,醫院 治安良好—近警察局住戶單純',
 '生活機能': '近便利商店、近醫療機構、近百貨公司、近傳統市場、近綠地公園、近夜市、近警察局、近火車站、近公車站、近捷運站、近學校',
 '管理費': '無',
 '設備': '傳統電視、第四台、脫水機、洗衣機、瓦斯熱水器、天然瓦斯、冰箱、冷氣',
 '身分要求': '學生、上班族',
 '開伙': '不接受',
 '隔間': '水泥牆'}

In [6]:
main = {}
for vv in soup.select('div.search_infobg_left3 > ul')[0].select('li'):
    kv = vv.text.strip().replace(' ','').replace('\n', ' ').split('：')
    if len(kv) == 2:
        main[kv[0]] = kv[1]

if soup.select('input#GPSLatitude')[0]['value'] != '':
    main['lat'] = float(soup.select('input#GPSLatitude')[0]['value'])
if soup.select('input#GPSLongitude')[0]['value'] != '':
    main['lng'] = float(soup.select('input#GPSLongitude')[0]['value'])

main


IndexError: list index out of range